# 1. Introduction
### To get a first feel of the Mcity Data Engine, we provide an online demo in a Google Colab environment. 
#### We will load the Fisheye8K dataset and demonstrate the Mcity Data Engine workflow Embedding Selection. This workflow leverages a set of models to compute image embeddings. After Voxel51 app is launched, the user can explore images using embedding space to identify the dataset for training data.

## Welcome to Mcity data engine in Google Colab.

## You have 2 options to choose:

### <b>Option 1</b> Run commands step by step

#### Go to <b>2.Step-by-Step Intructions </b> below

### <b>Option 2</b> Run All

#### Go to Header Menu-> Click on Runtime and select Run All (as per the below image) for the outcome:


<b> Chrome's Appearance Mode is Dark </b>

![image](https://github.com/user-attachments/assets/3ff8c429-6644-4f40-8cd2-3f570eeed089)



# 2. Step-by-Step Instructions

## ⚠️ Cell output is disabled by default. To print cell output, remove the <b>%%capture</b> and rerun ⚠️

### <b>Step 1</b>: Clone the mcity_data_engine GitHub repository using below command. This command will remove the existing folder (if any) and clone it from GitHub. This will make a copy of the code in the current Google Colab session.

In [ ]:
%%capture
!rm -rf mcity_data_engine && git clone https://github.com/mcity/mcity_data_engine.git

### <b>Step 2</b>: Copy the local copy of the code [from Step 1] to workspace. 

In [ ]:
!cp -R mcity_data_engine/* .

### <b>Step 3</b>: To execute embedding workflow, some configuration changes required. Same will be done by below script for Colab workflow:

In [ ]:
%%capture
!pip install typed-ast
!python update_config.py

### <b>Step 4</b>: Install Colab specific data engine requirements for this exercise.
#### <b>⚠️ It will take 5-6 minutes to install requirements ⚠️</b>

In [ ]:
%%capture
!pip install -r requirements_colab.txt

### <b>Step 5</b>: Configure Huggingface timeout variables to avoid session timeouts due to default timeout values.

In [ ]:
!export HF_HUB_ETAG_TIMEOUT=5000
!export HF_HUB_DOWNLOAD_TIMEOUT=1000

### <b>Step 6</b>: This exercise uses an open-source dataset named Voxel51/fisheye8k available on Hggingface,  which will be downloaded in this step. The Max_samples parameter is set to 100 (images from the dataset) for fast processing, but it can be increased to 8000 if needed.

In [ ]:
%%capture
from fiftyone.utils.huggingface import load_from_hub
import fiftyone as fo

if fo.dataset_exists("fisheye8k"):
  fo.delete_dataset("fisheye8k")

dataset = load_from_hub(
    "Voxel51/fisheye8k",
    name = "fisheye8k",
    max_samples=100,
    )

### <b>Step 7</b>: Start embedding workflow.

In [ ]:
!python main.py &> /dev/null

### <b>Step 8</b>: Set up the Voxel51 app layout for the dataset for which embeddings are computed. After Voxel51 app is launched, the user can explore images using embedding space to identify the dataset for training data. 

   ### Below are the key fields related to embedding, which you will find on the left filter once the Voxel51 app is opened:

- <b> embedding_selection --> </b> Type of selection (stages of the internal selection process)

- <b> embedding_selection_model --> </b> Which embedding model was responsible for the first selection

- <b> embedding_selection_count --> </b> How often it was selected

  ![image](https://github.com/user-attachments/assets/3c8c46d8-0545-4dc5-aa7c-09f319aca450)

In [ ]:
import fiftyone as fo
samples_panel = fo.Panel(type="Samples", pinned=True)

embeddings_panel = fo.Panel(
    type="Embeddings",
    state=dict(brainResult="clip_vit_base32_torch_umap", colorByField="embedding_selection_count"),
)

spaces = fo.Space(
    children=[
                fo.Space(children=[samples_panel], active_child=samples_panel.component_id),
                fo.Space(children=[embeddings_panel], active_child=embeddings_panel.component_id),
            ],
    orientation="horizontal",
)

session =fo.launch_app(dataset=dataset,spaces=spaces)

# 3. Handling Errors and Troubleshooting
* The Voxel51 app may take 1-2 minutes to launch, depending on your internet speed.
* If no images appear in the Samples Panel or Embeddings Panel, manually click on the Samples or Embeddings tab to load them (refer Voxel51 [Tutorial](https://docs.voxel51.com/tutorials/image_embeddings.html)).

# 4. Additional Notes and Best Practices
* Always ensure your runtime is set to GPU for optimal performance.
* Keep an eye on execution logs for warnings or errors.
* Save your work frequently to avoid losing progress.


By following this manual, you will be able to execute the Fish Eye 8K notebook in Google Colab efficiently. Happy coding!